In [ ]:
# Experience Cleaning

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("../Data/Raw/naukri_com-job_sample.csv")

# --- SAFETY RESET (removes any broken intermediate columns) ---
df = df.drop(columns=["Exp_Min", "Exp_Max", "Exp_Avg"], errors="ignore")

# --- EXTRACT EXPERIENCE RANGE ---
exp_range = df["experience"].astype(str).str.extract(
    r"(\d+)\s*-\s*(\d+)"
)

# Rename extracted columns
exp_range.columns = ["Exp_Min", "Exp_Max"]

# --- ASSIGN AS PROPER SERIES ---
df["Exp_Min"] = exp_range["Exp_Min"]
df["Exp_Max"] = exp_range["Exp_Max"]

# --- CONVERT TO NUMERIC SAFELY ---
df["Exp_Min"] = pd.to_numeric(df["Exp_Min"], errors="coerce")
df["Exp_Max"] = pd.to_numeric(df["Exp_Max"], errors="coerce")

# --- CREATE AVERAGE EXPERIENCE ---
df["Exp_Avg"] = (df["Exp_Min"] + df["Exp_Max"]) / 2

# --- FINAL CHECK ---
# df[["experience", "Exp_Min", "Exp_Max", "Exp_Avg"]].head(10)

In [ ]:
# Salary Cleaning

In [ ]:
# df["payrate"].value_counts().head(15)

In [ ]:
# --- SAFETY RESET ---
df = df.drop(columns=["Salary_Min", "Salary_Max", "Salary_Avg"], errors="ignore")

# --- KEEP ONLY ANNUAL RANGE SALARIES (e.g. 2,00,000 - 4,00,000 P.A) ---
salary_range = df["payrate"].astype(str).str.extract(
    r"([\d,]+)\s*-\s*([\d,]+)\s*P\.A"
)

salary_range.columns = ["Salary_Min", "Salary_Max"]

# --- ASSIGN TO DF ---
df["Salary_Min"] = salary_range["Salary_Min"]
df["Salary_Max"] = salary_range["Salary_Max"]

# --- REMOVE COMMAS & CONVERT TO NUMERIC ---
df["Salary_Min"] = (
    df["Salary_Min"]
    .str.replace(",", "", regex=False)
    .astype(float)
)

df["Salary_Max"] = (
    df["Salary_Max"]
    .str.replace(",", "", regex=False)
    .astype(float)
)

# --- CREATE AVERAGE SALARY ---
df["Salary_Avg"] = (df["Salary_Min"] + df["Salary_Max"]) / 2

# --- SANITY CHECK ---
# df[["payrate", "Salary_Min", "Salary_Max", "Salary_Avg"]].head(10)

In [ ]:
# Job Location Cleaning

In [ ]:
# df["joblocation_address"].value_counts().head(15)

In [ ]:
# --- CREATE PRIMARY CITY COLUMN (SAFE) ---
df["City"] = (
    df["joblocation_address"]
    .dropna()                      # prevent NaN → "nan"
    .str.split(",")
    .str[0]
    .str.split("/")
    .str[0]
    .str.strip()
)

# --- NORMALIZE CITY VARIANTS ---
city_map = {
    "Bengaluru": "Bangalore",
    "Bengaluru/Bangalore": "Bangalore",
    "Bengaluru Bangalore": "Bangalore",
    "Hyderabad / Secunderabad": "Hyderabad",
    "Delhi/NCR(National Capital Region)": "Delhi",
    "Delhi NCR": "Delhi",
    "Noida/Greater Noida": "Noida",
    
}
df["City"] = df["City"].replace({
    "Mumbai Suburbs": "Mumbai",
    "Navi Mumbai": "Mumbai"
})

df["City"].value_counts(dropna=False).head(15)

df["City"] = df["City"].replace(city_map)

# --- CLEAN EXTRA SPACES ---
df["City"] = df["City"].str.replace(r"\s+", " ", regex=True)

# --- FINAL CHECK ---
# df["City"].value_counts(dropna=False).head(15)

In [ ]:
# STEP 3 OUTPUT — CLEANED DATASET ONLY

cleaned_cols = [
    "jobid",
    "company",
    "jobtitle",
    "industry",
    "joblocation_address",
    "City",
    "experience",
    "Exp_Min",
    "Exp_Max",
    "Exp_Avg",
    "payrate",
    "Salary_Min",
    "Salary_Max",
    "Salary_Avg",
    "skills",
    "postdate",
    "uniq_id"
]

cleaned_df = df[cleaned_cols].copy()

cleaned_df.to_csv(
    "../Data/Processed/cleaned_job_listings.csv",
    index=False
)
